In [1]:
# -*- coding: utf-8 -*-
import os
import numpy as np
import cv2
import dlib
import time
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Load dataset

In [11]:
data = pd.read_pickle('faces_all.pkl')
detector = dlib.get_frontal_face_detector()
p = "shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor (p)
face_rec_model_path = 'dlib_face_recognition_resnet_model_v1.dat'
facerec = dlib.face_recognition_model_v1(face_rec_model_path)

In [12]:
data.shape

(14430, 3)

In [4]:
def get_faces(file):
    """
        This function has the same functionality as in 2-Get_face_descriptor.ipynb
        It returns a pandas dataframe contains the face decriptors
    """
    faces = []
    image = cv2.imdecode(np.fromfile(file,dtype=np.uint8),-1)
    image = cv2. cvtColor (image, cv2.COLOR_BGR2RGB)
    gray = cv2. cvtColor (image, cv2.COLOR_RGB2GRAY)

    rects = detector (gray , 0)
    if len(rects) >=0:
        for rect in rects :
            shape = predictor (gray , rect )
            face_chip = dlib.get_face_chip(image, shape)
            face_descriptor = facerec.compute_face_descriptor(face_chip)
            item = [file, [rect.left(), rect.top(), rect.width(), rect.height()], list(face_descriptor)]
            faces.append(item)
    return pd.DataFrame(faces, columns=['file', 'rect', 'face_descriptor'])

In [5]:
def rec_person(row):
    """
        This function takes a file's path (better absolute path) as argument and returns the image's information with 
        the most likely match
    """
    max_d = 1000
    person = None
    for i in range(len(data)):
        f = data.iloc[i]
        #print(f)
        if f.file == row.file:
            continue
        #print(row.face_descriptor)
        d = np.linalg.norm(np.array(f.face_descriptor) - np.array(row.face_descriptor))
        if d < max_d:
            max_d = d
            person = f
    if max_d<0.6:
        return([max_d,person])
    else:
        return None

In [6]:
def rec_persons(row):
    """
        This function takes a file's path (better absolute path) as argument and returns all images with similar face.
        The result is a list of pandas series
    """
    persons = []
    for i in range(len(data)):
        f = data.iloc[i]
        #print(f)
        if f.file == row.file:
            continue
        #print(row.face_descriptor)
        d = np.linalg.norm(np.array(f.face_descriptor) - np.array(row.face_descriptor))
        if d < 0.3:
            persons.append(f)
    return persons

In [13]:
img = "D:\\Data2\\video2\\8.29张江人工智能岛\\29号\\摄像迟\\BPAV\\CLPR\\444_0370_01\\4.0.jpg" # Path to sample image for searching
img_info = get_faces(img)
img_info

,file,rect,face_descriptor
0,D:\Data2\video2\8.29张江人工智能岛\29号\摄像迟\BPAV\CLPR\...,"[910, 251, 259, 259]","[-0.060811690986156464, 0.1016613319516182, 0...."


In [14]:
for i in range(len(img_info)):
    print("For image: ", img_info.iloc[i]['file'],":")
    result = rec_persons(img_info.iloc[i])
    for j in range(len(result)):
        print(result[j]['file'])
    print()

For image:  D:\Data2\video2\8.29张江人工智能岛\29号\摄像迟\BPAV\CLPR\444_0370_01\4.0.jpg :
D://Data2\video2\8.29张江人工智能岛\29号\摄像迟\BPAV\CLPR\444_0368_01\0.0.jpg
D://Data2\video2\8.29张江人工智能岛\29号\摄像迟\BPAV\CLPR\444_0368_01\1.0.jpg
D://Data2\video2\8.29张江人工智能岛\29号\摄像迟\BPAV\CLPR\444_0368_01\2.0.jpg
D://Data2\video2\8.29张江人工智能岛\29号\摄像迟\BPAV\CLPR\444_0368_01\3.0.jpg
D://Data2\video2\8.29张江人工智能岛\29号\摄像迟\BPAV\CLPR\444_0368_01\5.0.jpg
D://Data2\video2\8.29张江人工智能岛\29号\摄像迟\BPAV\CLPR\444_0368_01\8.0.jpg
D://Data2\video2\8.29张江人工智能岛\29号\摄像迟\BPAV\CLPR\444_0370_01\0.0.jpg
D://Data2\video2\8.29张江人工智能岛\29号\摄像迟\BPAV\CLPR\444_0370_01\1.0.jpg
D://Data2\video2\8.29张江人工智能岛\29号\摄像迟\BPAV\CLPR\444_0370_01\2.0.jpg
D://Data2\video2\8.29张江人工智能岛\29号\摄像迟\BPAV\CLPR\444_0370_01\3.0.jpg
D://Data2\video2\8.29张江人工智能岛\29号\摄像迟\BPAV\CLPR\444_0370_01\4.0.jpg
D://Data2\video2\8.29张江人工智能岛\29号\摄像迟\BPAV\CLPR\444_0504_01\0.0.jpg
D://Data2\video2\8.29张江人工智能岛\29号\摄像迟\BPAV\CLPR\444_0504_01\1.0.jpg
D://Data2\video2\8.29张江人工智能岛\29号\摄像迟\BPAV\CLPR\44